In [11]:
# this will autoreload your packages (i.e. if you make a change to one of your .py files, you will not need to restart your notebook - your change is dynamically imported)
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd


In [2]:
import os

import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" # to be able to make async database calls from the notebook
os.environ["DJANGO_SETTINGS_MODULE"] = "fpl.settings"
django.setup()

In [5]:
df_players = pd.read_csv('df_players.csv')
df_teams = pd.read_csv('df_team.csv')
df_teams


,id,name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,1,Arsenal,5,1350,1365,1370,1370,1330,1360
1,2,Aston Villa,4,1160,1285,1140,1220,1180,1350
2,3,Bournemouth,3,1100,1100,1055,1130,1145,1075
3,4,Brentford,3,1100,1100,1110,1055,1090,1150
4,5,Brighton,3,1100,1210,1070,1180,1135,1240
5,6,Burnley,2,1045,1050,1050,1050,1040,1050
6,7,Chelsea,3,1125,1190,1080,1150,1170,1230
7,8,Crystal Palace,3,1070,1100,1080,1120,1060,1085
8,9,Everton,3,1095,1100,1125,1125,1065,1080
9,10,Fulham,3,1055,1175,1050,1180,1060,1170


In [6]:
df = df_players[['first_name', 'second_name', 'team']].merge(df_teams[['name', 'id']], left_on = 'team', right_on = 'id')
df = df.drop(columns = ['team', 'id'])
df = df.rename(columns = {"name" : "team"})
df

,first_name,second_name,team
0,Aaron,Ramsdale,Arsenal
1,Rúnar Alex,Rúnarsson,Arsenal
2,David,Raya Martin,Arsenal
3,Karl,Hein,Arsenal
4,Emiliano,Martínez Romero,Aston Villa
...,...,...,...
832,Fábio,Silva,Wolves
833,Sasa,Kalajdzic,Wolves
834,Matheus,Santos Carneiro Da Cunha,Wolves
835,Nathan,Fraser,Wolves


In [27]:
from fpl.models import Club

for index, row in df_teams.iterrows():
    c = Club(name=row['name'])
    c.save()
    
    

In [7]:
from fpl.models import Player

for index, row in df.iterrows():
    p = Player(first_name=row['first_name'], last_name = row['second_name'], club = Club.objects.get(name = row['team']))
    print(p)
    break
    # p.save()

NameError: name 'Club' is not defined

In [2]:
import matplotlib.pyplot as plt

In [3]:
import matplotlib.pyplot as plt
import base64
from io import BytesIO

def get_graph():
    buffer = BytesIO()
    plt.savefig(buffer, format = 'png')
    buffer.seek(0)
    image_png = buffer.getvalue()
    graph = base64.b64encode(image_png)
    graph = graph.decode('utf-8')
    buffer.close()
    return graph

def get_plot(x,y):
    plt.switch_backend('AGG')
    plt.figure(figsize=(10,5))
    plt.title('Title')
    plt.plot(x,y)
    plt.xticks(rotation = 45)
    plt.xlabel('XLABEL')
    plt.ylabel('YLABEL')
    plt.tight_layout
    graph = get_graph()
    return graph

In [4]:
def main_view(request):
    qs = OverallStatistics.objects.all()
    x = [x.gameweek for x in qs]
    y = [y.average_score for y in qs]
    chart = get_plot(x, y)
    return render(request, 'overall.html', {'chart': chart})